# Déployez un modèle dans le cloud



### 4.10.1 Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1675675973171_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des informations sur la session en cours et liens vers Spark UI</u> :

### 4.10.2 Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

### 4.10.3 Import des librairies

In [2]:
import pandas as pd
import numpy as np
import io
import os

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from PIL import Image

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import tensorflow as tf

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [6]:
PATH = 's3://oc-lolo-p8-data'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://oc-lolo-p8-data
PATH_Data:   s3://oc-lolo-p8-data/Test
PATH_Result: s3://oc-lolo-p8-data/Results

### 4.10.5 Traitement des données

#### 4.10.5.1 Chargement des données

In [7]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://oc-lolo-p8-d...|2023-01-30 23:17:27|  7353|[FF D8 FF E0 00 1...|
|s3://oc-lolo-p8-d...|2023-01-30 23:17:27|  7350|[FF D8 FF E0 00 1...|
|s3://oc-lolo-p8-d...|2023-01-30 23:17:27|  7349|[FF D8 FF E0 00 1...|
|s3://oc-lolo-p8-d...|2023-01-30 23:17:27|  7348|[FF D8 FF E0 00 1...|
|s3://oc-lolo-p8-d...|2023-01-30 23:17:43|  7328|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [9]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------+----------+
|path                                              |label     |
+--------------------------------------------------+----------+
|s3://oc-lolo-p8-data/Test/Watermelon/r_106_100.jpg|Watermelon|
|s3://oc-lolo-p8-data/Test/Watermelon/r_109_100.jpg|Watermelon|
|s3://oc-lolo-p8-data/Test/Watermelon/r_108_100.jpg|Watermelon|
|s3://oc-lolo-p8-data/Test/Watermelon/r_107_100.jpg|Watermelon|
|s3://oc-lolo-p8-data/Test/Watermelon/r_95_100.jpg |Watermelon|
+--------------------------------------------------+----------+
only showing top 5 rows

None

#### 4.10.5.2 Préparation du modèle

In [10]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [11]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [14]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4.10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [15]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### 4.10.5.4 Exécutions des actions d'extractions de features

In [14]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://oc-lolo-p8-data/Results

In [18]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.6 Chargement des données enregistrées et validation du résultat

In [22]:
df = features_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# standardization with StandardScaler and import PCA in pysaprk
from pyspark.ml.feature import StandardScaler, PCA
from pyspark.ml.functions import array_to_vector

df = df.withColumn('features', array_to_vector('features'))

scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df)

# transform the dataframe
df_scaled = scaler.transform(df)
df_scaled.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+--------------------+
|                path|         label|            features|      scaledFeatures|
+--------------------+--------------+--------------------+--------------------+
|s3://oc-lolo-p8-d...|    Watermelon|[0.0,0.4021252989...|[-0.7574006047611...|
|s3://oc-lolo-p8-d...|Pineapple Mini|[0.0,4.0801434516...|[-0.7574006047611...|
|s3://oc-lolo-p8-d...|    Watermelon|[0.30721887946128...|[-0.1881095124504...|
|s3://oc-lolo-p8-d...|    Watermelon|[8.36627150420099...|[-0.7558502951410...|
|s3://oc-lolo-p8-d...|    Watermelon|[0.11927637457847...|[-0.5363758497405...|
+--------------------+--------------+--------------------+--------------------+
only showing top 5 rows

In [26]:
# fit columns 'scaledFeatures', 2 principal components 
pca = PCA(k=2, inputCol="scaledFeatures", outputCol="pcaFeatures")
model_pca = pca.fit(df_scaled.select('scaledFeatures'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Transformation 
df_pca_transformed = model_pca.transform(df_scaled)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# show new dataframe 
df_pca_transformed.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+--------------------+
|                path|     label|            features|      scaledFeatures|         pcaFeatures|
+--------------------+----------+--------------------+--------------------+--------------------+
|s3://oc-lolo-p8-d...|Watermelon|[0.84154003858566...|[0.80201282917138...|[-13.578741245307...|
|s3://oc-lolo-p8-d...|Watermelon|[0.0,0.4021252989...|[-0.7574006047611...|[-5.6778863538242...|
|s3://oc-lolo-p8-d...|Watermelon|[0.02070611529052...|[-0.7190311952641...|[-8.3709237392439...|
|s3://oc-lolo-p8-d...|Watermelon|[0.18416415154933...|[-0.4161357346339...|[-13.478061288805...|
|s3://oc-lolo-p8-d...|Watermelon|[0.41570547223091...|[0.01292126732189...|[-13.679494461093...|
+--------------------+----------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [29]:
#save as json format
df_pca_transformed.write.mode("overwrite").json(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_result = spark.read.json(PATH_Result + '/*.json')
df_result.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+--------------------+--------------------+--------------------+
|            features|         label|                path|         pcaFeatures|      scaledFeatures|
+--------------------+--------------+--------------------+--------------------+--------------------+
|{1, [1.1428744792...|    Watermelon|s3://oc-lolo-p8-d...|{1, [-14.34861957...|{1, [1.3603997785...|
|{1, [1.3194596767...|    Watermelon|s3://oc-lolo-p8-d...|{1, [-14.62411263...|{1, [1.6876204888...|
|{1, [0.0401640757...|Pineapple Mini|s3://oc-lolo-p8-d...|{1, [-13.51770216...|{1, [-0.682974675...|
|{1, [0.6248898506...|    Watermelon|s3://oc-lolo-p8-d...|{1, [-10.35058049...|{1, [0.4005497993...|
|{1, [0.0359662100...|    Watermelon|s3://oc-lolo-p8-d...|{1, [-10.87490829...|{1, [-0.690753518...|
|{1, [0.0, 5.16979...|Pineapple Mini|s3://oc-lolo-p8-d...|{1, [-13.26912349...|{1, [-0.757400604...|
|{1, [0.2590608894...|     Raspberry|s3://oc-lolo-p8-d...|{1, [-2.170062305...|{1, [-0.2773